This is the first part of the quick start series of tutorials about Hopsworks Feature Store. As part of this first module, you will work with data related to credit card transactions. 
The objective of this tutorial is to demonstrate how to work with the **Hopworks Feature Store**  for online data with a goal of training and deploying a model that can predict fraudulent transactions.

## <span style='color:#ff5f27'> 📝 Imports

In [ ]:
import hopsworks

import great_expectations as ge
from great_expectations.core import ExpectationSuite, ExpectationConfiguration

from mlopstemplate.features import transactions, profile
from mlopstemplate.synthetic_data import synthetic_data
from mlopstemplate.synthetic_data.data_sources import get_datasets

First of all you will load the data and do some feature engineering on it.

## <span style="color:#ff5f27;"> 💽 Loading the Data </span>


In [ ]:
# get data from the source
trans_df, labels_df, profiles_df = get_datasets()

---

## <span style="color:#ff5f27;"> 🛠️ Feature Engineering </span>


Now you are ready to start by computing the distance between consecutive transactions, lets call it `loc_delta`.
Here you will use the [Haversine distance](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.pairwise.haversine_distances.html?highlight=haversine#sklearn.metrics.pairwise.haversine_distances) to quantify the distance between two longitude and latitude coordinates.

In [ ]:
# compute profile features
# select final features
profiles_df = profile.select_features(profiles_df)

In [ ]:
# Compute transaction features
trans_df = trans_df.sort_values(by=['cc_num', 'datetime'])
trans_df["prev_transaction_date"] = trans_df.groupby("cc_num").apply(lambda x: x["datetime"].shift(1)).reset_index(level=0, drop=True)
trans_df["prev_longitude"] = trans_df.groupby("cc_num").apply(lambda x: x["longitude"].shift(1)).reset_index(level=0, drop=True)
trans_df["prev_latitude"] = trans_df.groupby("cc_num").apply(lambda x: x["latitude"].shift(1)).reset_index(level=0, drop=True)
trans_df = trans_df.dropna()

# compute previous location of the transaction
trans_df["loc_delta_t_minus_1"] = transactions.haversine(trans_df.longitude, trans_df.latitude, trans_df.prev_longitude, trans_df.prev_latitude)

# Computes time difference between current and previous transaction
trans_df["time_delta_t_minus_1"] = transactions.time_delta_t_minus_1(trans_df.datetime, trans_df.prev_transaction_date)

# compute number of transactions per day
trans_df["year_month_day"] = trans_df.datetime.map(lambda x: str(x.year) + "-" + str(x.month) + "-" + str(x.day))
count_by_day = trans_df.groupby(['cc_num', 'year_month_day']).size().reset_index(name='number_of_transactions_daily')
trans_df = trans_df.merge(count_by_day, on=["cc_num", "year_month_day"], how="left")

# Compute year and month string from datetime column, for partition 
trans_df["month"] = transactions.get_year_month(trans_df.datetime)

# compute on demand features
# customer's age at transaction
trans_df = trans_df.merge(profiles_df[["cc_num", "birthdate", "cc_expiration_date"]], on=["cc_num"], how="left")

# age at transaction
trans_df["age_at_transaction"] = transactions.card_owner_age(trans_df.datetime, trans_df.birthdate)

# days untill card expires at the time of transaction
trans_df["days_until_card_expires"] = transactions.expiry_days(trans_df.datetime, trans_df.cc_expiration_date)

# select features
trans_df = transactions.select_features(trans_df)

In [ ]:
# labels
labels_df["month"] = transactions.get_year_month(labels_df.datetime)

## <span style="color:#ff5f27;"> Great Expectations </span>


In [ ]:
ge_trans_df = ge.from_pandas(trans_df)

expectation_suite_trans = ge_trans_df.get_expectation_suite()
expectation_suite_trans.expectation_suite_name = "transaction_suite"
print(expectation_suite_trans)


In [ ]:
# Check for errors which could lead to technical issues
expectation_suite_trans.add_expectation(
    ExpectationConfiguration(
        expectation_type="expect_column_values_to_be_unique",
        kwargs={"column":"tid", "result_format":"COMPLETE"}
    )
)

# Assess data correctness
expectation_suite_trans.add_expectation(
    ExpectationConfiguration(
        expectation_type="expect_column_min_to_be_between",
        kwargs={
            "column":"amount",
            "min_value": 0
        }
    )
)

expectation_suite_trans.add_expectation(
    ExpectationConfiguration(
        expectation_type="expect_column_min_to_be_between",
        kwargs={
            "column":"age_at_transaction",
            "min_value": 17,
            "max_value": 101
        }
    )
)

ge_trans_df = ge.from_pandas(trans_df, expectation_suite=expectation_suite_trans)

validation_report_trans = ge_trans_df.validate()

In [ ]:
validation_report_trans

In [ ]:
#Generate an Expectation Suite from your data using Great Expectations Profiler
#For complex DataFrames, Great Expectations offers a profiler which generates a basic expectation suite tailored to your data. You can then use this suite as you would any other expectation suite with Hopsworks.

# ignore deprecation warnings
expectation_suite_profiled, validation_report = ge_trans_df.profile(profiler=ge.profile.BasicSuiteBuilderProfiler)

print(f"The suite contains {len(expectation_suite_profiled['expectations'])} expectations for {len(trans_df.columns.values)} columns. See sample below\n" + ge_trans_df.get_expectation_suite().__repr__()[:455])

#Note that you cannot register the report generated by the profiler as the suite was not registered with Hopsworks before the validation was run.

## <span style="color:#ff5f27;"> 📡 Connecting to Hopsworks Feature Store </span>

### <span style="color:#ff5f27;"> 🪄 Creating Feature Groups </span>

A [feature group](https://docs.hopsworks.ai/3.0/concepts/fs/feature_group/fg_overview/) can be seen as a collection of conceptually related features. In this case, you will create a feature group for the transaction data and a feature group for the windowed aggregations on the transaction data. Both will have `cc_num` as primary key, which will allow you to join them when creating a dataset in the next tutorial.

Feature groups can also be used to define a namespace for features. For instance, in a real-life setting you would likely want to experiment with different window lengths. In that case, you can create feature groups with identical schema for each window length. 

Before you can create a feature group you need to connect to Hopsworks feature store.

In [ ]:
import hopsworks

project = hopsworks.login()
fs = project.get_feature_store()

To create a feature group you need to give it a name and specify a primary key. It is also good to provide a description of the contents of the feature group and a version number, if it is not defined it will automatically be incremented to `1`.

In [ ]:
# get or create feature group
trans_fg = fs.get_or_create_feature_group(
    name="transactions",
    version=1,
    description="Transaction data",
    primary_key=['cc_num'],
    event_time='datetime',
    partition_key=['month'],
    stream=True,
    online_enabled=True
)

In [ ]:
# 4.Setup Automatic Validation On Insert and upload a DataFrame
# Register the expectation suite corresponding to a Feature Group with the backend

# The "ALWAYS" ingestion policy inserts data even when validation fails, 
# ideal to avoid data loss and rapid prototyping

trans_fg.save_expectation_suite(expectation_suite_trans, validation_ingestion_policy="ALWAYS")

#Once the suite is registered in the backend, data validation will run on every insert without additional boilerplate. The suite is retrieved from the backend, used to validate the DataFrame and the resulting validation report uploaded. Depending on the ingestion policy and validation success, data are subsequently inserted in the Feature Group. The example below illustrate the "ALWAYS" use case where insertion is performed despite a validation failure.

# materialize feature data in to the feature group
trans_fg.insert(trans_df)

Here you have also set `online_enabled=True`, which enables low latency access to the data. A full list of arguments can be found in the [documentation](https://docs.hopsworks.ai/feature-store-api/latest/generated/api/feature_store_api/#create_feature_group).

In [ ]:
feature_descriptions = [
    {"name": "tid", "description": "Transaction id"},
    {"name": "datetime", "description": "Transaction time"},
    {"name": "cc_num", "description": "Number of the credit card performing the transaction"},
    {"name": "amount", "description": "Month of transaction"},
    {"name": "year_month_day", "description": "Day of transaction"},
    {"name": "latitude", "description": "latitude of the place where merchant or ATM is located"},
    {"name": "longitude", "description": "longitude of the place where merchant or ATM is located"},
    {"name": "country", "description": "Country in which the transaction was made"},
    {"name": "loc_delta_t_minus_1", "description": "Location of previous transaction"},
    {"name": "time_delta_t_minus_1", "description": "Time of previous transaction"},    
    {"name": "month", "description": "Month of the transaction"},    
    {"name": "age_at_transaction", "description": "Age of card holder at the time of transaction"},
    {"name": "days_until_card_expires", "description": "Days left till card expires"},
    {"name": "number_of_transactions_daily", "description": "Number of transactions per cc daily"},
]

for desc in feature_descriptions: 
    trans_fg.update_feature_description(desc["name"], desc["description"])

In [ ]:
# get or create feature group
labels_fg = fs.get_or_create_feature_group(
    name="fraud_labels",
    version=1,
    description="Transaction data",
    primary_key=['cc_num'],
    event_time='datetime',
    partition_key=['month'],
    stream=True,
    online_enabled=True
)

In [ ]:
# materialize feature data in to the feature group
labels_fg.insert(labels_df)

You can move on and do the same thing for the profile and label feature groups.

In [ ]:
# get or create feature group
profile_fg = fs.get_or_create_feature_group(
    name="profile",
    version=1,
    description="Credit card holder demographic data",
    primary_key=["cc_num"],
    partition_key=["cc_provider"],
    stream=True,
    online_enabled=True
)


In [ ]:
# materialize feature data in to the feature group
profile_fg.insert(profiles_df)

In [ ]:
feature_descriptions = [
    {"name": "cc_num", "description": "Number of the credit card performing the transaction"},
    {"name": "cc_provider", "description": "Company name that issued the card"},
    {"name": "cc_type", "description": "Type of the card, debit or credit"},
    {"name": "cc_expiration_date", "description": "Date when this card expires"},    
    {"name": "birthdate", "description": "Birth date"},
    {"name": "country_of_residence", "description": "Country of residence of the card holder"},    
]

for desc in feature_descriptions: 
    profile_fg.update_feature_description(desc["name"], desc["description"])

Click on the hyperlink printed in the cell output above to inspect your feature group in the UI.

### <span style="color:#ff5f27;"> 👓  Exploration</span>
In the Hopsworks feature store, the metadata allows for multiple levels of explorations and review. Here you will explore a few of those capacities. 

### <span style="color:#ff5f27;"> 🔎 Search</span>
Using the search function in the UI, you can query any aspect of the feature groups, feature_view and training data that was previously created.

### <span style="color:#ff5f27;"> 📊 Statistics</span>
You can also enable statistics in one or all the feature groups.

In [ ]:
trans_fg = fs.get_or_create_feature_group("transactions", version=1)
trans_fg.statistics_config = {
    "enabled": True,
    "histograms": True,
    "correlations": True
}

trans_fg.update_statistics_config()
trans_fg.compute_statistics()

### ⛓️ <b> Lineage </b> 
In all the feature groups and feature view you can look at the relation between each abstractions; what feature group created which training dataset and that is used in which model.
This allows for a clear undestanding of the pipeline in relation to each element. 